In [1]:
import pandas as pd
import requests
import json

In [2]:
r = requests.get("https://api.covid19india.org/states_daily.json")

In [3]:
data = pd.DataFrame(r.json()["states_daily"])

In [4]:
data.columns = data.columns.str.upper()

## Changing data type to int for interger value columns 

In [5]:
cols =  list(data.columns[:7]) + list(data.columns[8:32]) + list(data.columns[33:])

In [6]:
for i in cols:
    data[i] = data[i].astype("int64")

In [7]:
#data

Data for Daman and Diu, Dadar and Nagar Haveli is merged as its being reported together going further.

In [8]:
data["DNDD"] = data["DD"] + data["DN"]

In [9]:
#data["DNDD"]

In [10]:
data.drop(columns = ["DN", "DD"], inplace = True )

In [11]:
#data.info()

In [12]:
#data

## Creating time series data

In [13]:
data_ts = data.copy()

In [14]:
#data_ts

In [15]:
data_ts["DATE"] = data_ts["DATE"].astype("datetime64")

In [16]:
#data_ts.DATE

In [17]:
data_ts = data_ts.set_index(["STATUS","DATE"]).groupby(["STATUS"]).cumsum(axis = 0).unstack().T#.reset_index()#

In [18]:
#data_ts.sort_index(ascending = True)


In [19]:
data_ts.reset_index(inplace = True)

In [20]:
data_ts.rename(columns = {"level_0":"State"}, inplace = True)

There are some cases unassigned to any state, so captured separately.

In [21]:
data_ts.State.fillna("UN",inplace = True)

In [22]:
state_names_map = {"MH":"Maharashtra","TN":"Tamil Nadu","DL":"Delhi","GJ":"Gujarat","UP":"Uttar Pradesh","RJ":"Rajasthan",
                   "WB":"West Bengal","MP":"Madhya Pradesh","HR":"Haryana","KA":"Karnataka","AP":"Andhra Pradesh","BR":"Bihar",
                   "TG":"Telangana","JK":"Jammu and Kashmir","AS":"Assam","OR":"Odisha","PB":"Punjab","KL":"Kerala",
                   "UT":"Uttarakhand","CT":"Chhattisgarh","JH":"Jharkhand","TR":"Tripura","GA":"Goa","LA":"Ladakh",
                   "MN":"Manipur","HP":"Himachal Pradesh","CH":"Chandigarh","PY":"Puducherry","NL":"Nagaland",
                   "MZ":"Mizoram","AR":"Arunachal Pradesh","SK":"Sikkim", "DNDD":"Dadra and Nagar Haveli and Daman and Diu",
                   "AN": "Andaman and Nicobar Islands","ML":"Meghalaya","LD":"Lakshadweep","UN":"Unknown","TT":"Total"}

In [23]:
data_ts.State = data_ts.State.map(state_names_map)

In [24]:
data_ts["Active"] = data_ts.Confirmed - (data_ts.Deceased + data_ts.Recovered)

Country column added for Tableau visualization

In [25]:
data_ts.insert(loc = 0, column = "Country", value = "India"  )

In [26]:
#data_ts

In [27]:
data_ts.to_excel("TS_data.xlsx",index = False)

## Aggregating data

In [28]:
data_agg = data.groupby("STATUS").sum()

In [29]:
#data_agg

In [30]:
data_agg.drop(columns = "TT", inplace = True)

In [31]:
data_agg = data_agg.T

In [32]:
data_agg.sort_values( by = "Confirmed", ascending = False, inplace = True)

In [33]:
data_agg = data_agg[["Confirmed", "Recovered" , "Deceased" ]]

In [34]:
data_agg.columns.name = data_agg.columns.name.title()

In [35]:
#data_agg

In [36]:
data_agg.insert(loc = 1, column = "Active", value = data_agg.Confirmed - (data_agg.Recovered + data_agg.Deceased))

## Reading testing data (statewise) from API

In [37]:
r = requests.get("https://api.covid19india.org/state_test_data.json")
d = pd.DataFrame(r.json()["states_tested_data"])

In [38]:
d = d.loc[:, ["state","totaltested"]]

In [39]:
min_val = d.totaltested.min()

In [40]:
d.loc[d.totaltested == min_val] = d.loc[d.totaltested == min_val].replace(min_val, 0)

In [41]:
d["totaltested"] = d["totaltested"].astype("int64")

In [42]:
data_test = d.groupby("state").totaltested.max().to_frame()

In [43]:
#data_test

## Creating consolidated data (State wise)

In [44]:
data_agg.index = data_agg.index.map(state_names_map)

In [45]:
data_con = data_agg.join(data_test, how = "left").fillna(0)

In [46]:
#data_con.info()

In [47]:
data_con.totaltested = data_con.totaltested.astype("int64")

In [48]:
data_con.loc["TT"] = data_con.sum(axis = 0)

In [49]:
#data_con

In [50]:
data_con["Infection_Rate"] = (data_con["Confirmed"].div(data_con["totaltested"])).mul(100)

In [51]:
data_con["Recovery_Rate"] = (data_con["Recovered"].div(data_con["Confirmed"])).mul(100)

In [52]:
data_con["Mortality_Rate"] = (data_con["Deceased"].div(data_con["Confirmed"])).mul(100)

In [53]:
data_con.index.name = "States"

In [54]:
data_con.rename(index = {"TT":"Total"}, inplace = True)

In [55]:
data_con.reset_index(inplace = True)

In [56]:
#data_con


In [57]:
data_con.loc[data_con.States == "Unknown","Infection_Rate"] = 0

In [58]:
data_con.fillna(0,inplace = True)

In [59]:
data_con.to_csv("Covid19_India.csv", index = False)

### Data written into csv and excel files will be used to visualize it in Tableau

### URL for Tableau visualization: 

https://public.tableau.com/profile/shubham.rawat7488#!/vizhome/Covid19India/Dashboard1